In [1]:
import requests as req
import pandas as pd
from tqdm import tqdm

In [2]:
df=pd.read_csv('language_repo.csv')
df['repos']=df['f0_']
del df['f0_']
df.sort_values(by=['repos'],inplace=True,ascending=False)
df

,language_name,repos
10,JavaScript,1109031
78,CSS,822575
25,HTML,786975
6,Shell,641566
11,Python,551354
...,...,...
383,YAML,2
349,JSON,1
377,ZAP,1
379,Org,1


In [3]:
#languages that we dont consider to be general programming languages
not_languages=['CSS','HTML','Shell','Makefile','Batchfile','ApacheConf','CMake','PowerShell','XSLT','TeX']
#lets remove them from our list and just keep top 17 
#seems like a arbitrary number though
df=df[~df['language_name'].isin(not_languages)][:17]

df

,language_name,repos
10,JavaScript,1109031
11,Python,551354
12,Ruby,377741
9,Java,371385
7,PHP,343244
0,C,294270
51,C++,280535
4,Objective-C,170012
41,C#,133876
8,Perl,102809


In [4]:
df['repos'].quantile([0.25,0.5,0.75])

0.25     48245.0
0.50    133876.0
0.75    343244.0
Name: repos, dtype: float64

In [6]:
# Create target Directory if don't exist
import os
dirName='language_stats'
if not os.path.exists(dirName):
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ")
else:    
    print("Directory " , dirName ,  " already exists")

Directory  language_stats  Created 


In [7]:
#fetch top projects by stars
repo_count=50
client_id='insert_client_id_here'
client_secret='inser_client_secret_here'

for language in tqdm(df['language_name']):
    f = {'project_name': [], 'stargazers_count': []}
    r = req.get('https://api.github.com/search/repositories?q=language:'+req.compat.quote_plus(str(language))+'&sort=stars&order=desc&per_page='+str(repo_count)+'&client_id='+str(client_id)+'&client_secret='+str(client_secret))
    for obj in r.json()['items']:
        try:
            f['project_name'].append(str(obj['full_name']))
            f['stargazers_count'].append(str(obj['stargazers_count']))
        except:
            print("Couldnt find stargazer count")
        pd.DataFrame(data=f).to_csv('language_stats/'+str(language)+'.csv',index=False)

100%|██████████| 17/17 [01:33<00:00,  5.48s/it]


In [8]:
from glob import glob
d = {'Language': [], 'Projects': []}

In [9]:
for language in df['language_name']:
    try:
        temp=pd.read_csv('language_stats/'+language+'.csv')
        d['Language'].append(language)
        d['Projects'].append(' , '.join(temp[:3].values[:,0]))
    except:
        print("DONE")

In [10]:
pd.DataFrame(data=d)

,Language,Projects
0,JavaScript,"freeCodeCamp/freeCodeCamp , vuejs/vue , facebo..."
1,Python,"donnemartin/system-design-primer , vinta/aweso..."
2,Ruby,"rails/rails , jekyll/jekyll , discourse/discourse"
3,Java,"CyC2018/CS-Notes , Snailclimb/JavaGuide , iluw..."
4,PHP,"laravel/laravel , blueimp/jQuery-File-Upload ,..."
5,C,"torvalds/linux , netdata/netdata , antirez/redis"
6,C++,"tensorflow/tensorflow , electron/electron , mi..."
7,Objective-C,"AFNetworking/AFNetworking , SDWebImage/SDWebIm..."
8,C#,"shadowsocks/shadowsocks-windows , CodeHubApp/C..."
9,Perl,"so-fancy/diff-so-fancy , brendangregg/FlameGra..."


In [11]:
pd.DataFrame(data=d).to_csv('table1.csv',index=False)